### Assignment 1

Finalize Exercise 11.1 in `Week6/11_Full_MatSci-ML_Pipeline.ipynb`. Please, fill in the details and results in this [Google Sheet](https://docs.google.com/spreadsheets/d/1xbT4lRMYQGrhBQGFczFD5wscrriwWQGf82Gv9AnDkbA/).

In [ ]:
from matminer.datasets.dataset_retrieval import load_dataset, get_all_dataset_info
import pandas as pd
import matplotlib.pyplot as plt
from pymatgen.core import Composition
from matminer.featurizers.composition.composite import ElementProperty, ElementFraction
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from tqdm.notebook import tqdm
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score

print(get_all_dataset_info('citrine_thermal_conductivity'))

# dataset analysis

df = load_dataset('citrine_thermal_conductivity')
df.describe()

# Remove the highest thermal conductivity data point (potential outlier)
max_k = df['k_expt'].max()
df = df[df['k_expt'] < max_k]
print(f"Removed data point with highest k_expt = {max_k:.3f} W/mK")
print(f"Dataset now has {len(df)} entries.")


df['k_expt'].hist(bins=15)
plt.xlabel('Experimentally Measured Thermal Conductivity (W/mK)')
plt.ylabel('Frequency')
plt.title('Histogram of Thermal Conductivity')
plt.show()

# composition-based features- ElementProperty

df_comp = df.copy()
df_comp['composition'] = df_comp['formula'].apply(lambda x: Composition(x) )
df_comp.head()

# featurization- ElementProperty

el_prop_featuriser = ElementProperty.from_preset(preset_name='magpie', impute_nan=False)
el_prop_featuriser.set_n_jobs(1)
df_featurised = el_prop_featuriser.featurize_dataframe(df_comp, col_id='composition')

print(df_featurised.shape)
print(df_featurised.isnull().sum().sum())
df_featurised.head()

# feature engineering and cleaning

y = df_featurised['k_expt']
X_all = df_featurised.drop(columns=['k_expt', 'formula', 'k-units', 'k_condition', 'k_condition_units', 'composition'])

print("Number of features before cleaning:", X_all.shape[1])

# identify columns with very small variance and drop them
small_var_cols = X_all.columns[X_all.var() < 1e-5].tolist()
print("Columns with very small variance:", small_var_cols)
X_all = X_all.drop(columns = small_var_cols)
corr_matrix = X_all.corr(method='pearson')
print("Number of features after removing small variance columns:", X_all.shape[1])

# remove highly correlated columns
threshold = 0.99
to_drop = set()
for col in corr_matrix.columns:
    high_corr = corr_matrix.index[(corr_matrix[col].abs() > threshold) & (corr_matrix.index != col)]
    to_drop.update(high_corr)
print("Columns to drop due to high correlation:", to_drop)
X = X_all.drop(columns=list(to_drop))
print("Number of features after removing highly correlated columns:", X.shape[1])

#visualizing correlation matrix for remaining features
corr_matrix = X.corr(method='pearson')
plt.figure(figsize=(16,12))
im = plt.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title('Pearson Correlation Matrix')
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90, fontsize=6)
plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns, fontsize=6)
plt.tight_layout()
plt.show()

# feature scaling and dataset splitting

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

test_fraction = 0.1
validation_fraction = 0.2
X_trainval, X_test, y_trainval, y_test = train_test_split(X_scaled, y,
                                                          test_size=test_fraction,
                                                          random_state=3)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval,
                                                  test_size=validation_fraction/(1-test_fraction),
                                                  random_state=3)
print(f"Training fraction: {X_train.shape[0] / X_scaled.shape[0]:.2f}")
print(f"Validation fraction: {X_val.shape[0] / X_scaled.shape[0]:.2f}")
print(f"Test fraction: {X_test.shape[0] / X_scaled.shape[0]:.2f}")

# dummy baseline model

mean_train = y_train.mean()
baseline_mae = mean_absolute_error(y_val, [mean_train] * len(y_val))
print(f"Baseline MAE (predicting mean thermal conductivity): {baseline_mae:.4f} W/mK")

# hyperparameter optimization

n_estimators_list = [10, 50, 100, 500]
train_maes = []
val_maes = []

for n in tqdm(n_estimators_list, desc='Training RF models'):
    rf = RandomForestRegressor(n_estimators=n, random_state=3, n_jobs=1)
    rf.fit(X_train, y_train)
    y_train_pred = rf.predict(X_train)
    y_val_pred = rf.predict(X_val)
    train_maes.append(mean_absolute_error(y_train, y_train_pred))
    val_maes.append(mean_absolute_error(y_val, y_val_pred))

plt.figure(figsize=(8,5))
plt.plot(n_estimators_list, train_maes, marker='o', label='Train MAE')
plt.plot(n_estimators_list, val_maes, marker='o', label='Validation MAE')
plt.axhline(baseline_mae, color='gray', linestyle='--', label='Mean Baseline')
plt.xlabel('Number of Estimators')
plt.ylabel('Mean Absolute Error')
plt.title('Random Forest: MAE vs. Number of Estimators')
plt.legend()
plt.tight_layout()
plt.show()

# recursive feature elimination

rf = RandomForestRegressor(n_estimators=500, random_state=3, n_jobs=1)
rf.fit(X_train, y_train)

# Rank features by importance
importances = pd.Series(rf.feature_importances_, index=X.columns)
importances_sorted = importances.sort_values(ascending=False)

# Plot top 20 features by importance
plt.figure(figsize=(8,6))
importances_sorted.head(20).plot(kind='barh')
plt.gca().invert_yaxis()
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top 20 Most Important Features')
plt.tight_layout()
plt.show()

# Create dictionary to mimic RFE-style selection
selected_features_dict = {}
n_features_list = list(range(2, min(60, len(X.columns)), 2))  # up to 60 or total number of features
val_errors = []
train_errors = []

for n_features in tqdm(n_features_list, desc='Feature Importance Selection Progress'):
    selected = importances_sorted.head(n_features).index
    selected_features_dict[n_features] = list(selected)
    X_train_sel = X_train[:, [X.columns.get_loc(f) for f in selected]]
    X_val_sel = X_val[:, [X.columns.get_loc(f) for f in selected]]
    rf_temp = RandomForestRegressor(n_estimators=100, random_state=3, n_jobs=1)
    rf_temp.fit(X_train_sel, y_train)
    y_train_pred = rf_temp.predict(X_train_sel)
    y_val_pred = rf_temp.predict(X_val_sel)
    train_errors.append(mean_absolute_error(y_train, y_train_pred))
    val_errors.append(mean_absolute_error(y_val, y_val_pred))

plt.figure(figsize=(8,5))
plt.plot(n_features_list, train_errors, label='Train MAE')
plt.plot(n_features_list, val_errors, label='Validation MAE')
plt.xlabel('Number of Selected Features')
plt.ylabel('Mean Absolute Error')
plt.title('Feature Importance Selection: MAE vs. Number of Features')
plt.legend()
plt.tight_layout()
plt.show()

# final model evaluation

final_features = selected_features_dict[10] 
rf_final = RandomForestRegressor(n_estimators=500, random_state=3, n_jobs=1)
X_train_final = X_train[:, [X.columns.get_loc(f) for f in final_features]]
rf_final.fit(X_train_final, y_train)

# predict on train, validation, and test sets
X_val_final = X_val[:, [X.columns.get_loc(f) for f in final_features]]
X_test_final = X_test[:, [X.columns.get_loc(f) for f in final_features]]

y_train_pred = rf_final.predict(X_train_final)
y_val_pred = rf_final.predict(X_val_final)
y_test_pred = rf_final.predict(X_test_final)

# calculate metrics
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_val = mean_absolute_error (y_val, y_val_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_val = r2_score(y_val, y_val_pred)
r2_test = r2_score(y_test, y_test_pred)

fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharex=True, sharey=True)
min_val = 0.0
max_val = 450.0

# train parity plot
axes[0].scatter(y_train, y_train_pred, alpha=0.5, color='blue')
axes[0].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[0].set_title(f'Train (MAE={mae_train:.3f}, R2={r2_train:.3f})')
axes[0].set_xlabel('True Formation Energy')
axes[0].set_ylabel('Predicted Formation Energy')
axes[0].set_aspect('equal', adjustable='box')

# validation parity plot
axes[1].scatter(y_val, y_val_pred, alpha=0.5, color='orange')
axes[1].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[1].set_title(f'Validation (MAE={mae_val:.3f}, R2={r2_val:.3f})')
axes[1].set_xlabel('True Formation Energy')
axes[1].set_ylabel('Predicted Formation Energy')
axes[1].set_aspect('equal', adjustable='box')

# test parity plot
axes[2].scatter(y_test, y_test_pred, alpha=0.5, color='green')
axes[2].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[2].set_title(f'Test (MAE={mae_test:.3f}, R2={r2_test:.3f})')
axes[2].set_xlabel('True Formation Energy')
axes[2].set_ylabel('Predicted Formation Energy')
axes[2].set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

# featurization- ElementFraction

el_frac_featuriser = ElementFraction()
el_frac_featuriser.set_n_jobs(1)
df_featurised = el_frac_featuriser.featurize_dataframe(df_comp, col_id='composition')

print(df_featurised.shape)
print(df_featurised.isnull().sum().sum())
df_featurised.head()

# feature engineering and cleaning

y = df_featurised['k_expt']
X_all = df_featurised.drop(columns=['k_expt', 'formula', 'k-units', 'k_condition', 'k_condition_units', 'composition'])

print("Number of features before cleaning:", X_all.shape[1])

# identify columns with very small variance and drop them
small_var_cols = X_all.columns[X_all.var() < 1e-5].tolist()
print("Columns with very small variance:", small_var_cols)
X_all = X_all.drop(columns = small_var_cols)
corr_matrix = X_all.corr(method='pearson')
print("Number of features after removing small variance columns:", X_all.shape[1])

# remove highly correlated columns
threshold = 0.99
to_drop = set()
for col in corr_matrix.columns:
    high_corr = corr_matrix.index[(corr_matrix[col].abs() > threshold) & (corr_matrix.index != col)]
    to_drop.update(high_corr)
print("Columns to drop due to high correlation:", to_drop)
X = X_all.drop(columns=list(to_drop))
print("Number of features after removing highly correlated columns:", X.shape[1])

#visualizing correlation matrix for remaining features
corr_matrix = X.corr(method='pearson')
plt.figure(figsize=(16,12))
im = plt.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title('Pearson Correlation Matrix')
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90, fontsize=6)
plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns, fontsize=6)
plt.tight_layout()
plt.show()

# feature scaling and dataset splitting

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

test_fraction = 0.1
validation_fraction = 0.2
X_trainval, X_test, y_trainval, y_test = train_test_split(X_scaled, y,
                                                          test_size=test_fraction,
                                                          random_state=3)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval,
                                                  test_size=validation_fraction/(1-test_fraction),
                                                  random_state=3)
print(f"Training fraction: {X_train.shape[0] / X_scaled.shape[0]:.2f}")
print(f"Validation fraction: {X_val.shape[0] / X_scaled.shape[0]:.2f}")
print(f"Test fraction: {X_test.shape[0] / X_scaled.shape[0]:.2f}")

# dummy baseline model

mean_train = y_train.mean()
baseline_mae = mean_absolute_error(y_val, [mean_train] * len(y_val))
print(f"Baseline MAE (predicting mean thermal conductivity): {baseline_mae:.4f} W/mK")

# hyperparameter optimization

n_estimators_list = [10, 50, 100, 500]
train_maes = []
val_maes = []

for n in tqdm(n_estimators_list, desc='Training RF models'):
    rf = RandomForestRegressor(n_estimators=n, random_state=3, n_jobs=1)
    rf.fit(X_train, y_train)
    y_train_pred = rf.predict(X_train)
    y_val_pred = rf.predict(X_val)
    train_maes.append(mean_absolute_error(y_train, y_train_pred))
    val_maes.append(mean_absolute_error(y_val, y_val_pred))

plt.figure(figsize=(8,5))
plt.plot(n_estimators_list, train_maes, marker='o', label='Train MAE')
plt.plot(n_estimators_list, val_maes, marker='o', label='Validation MAE')
plt.axhline(baseline_mae, color='gray', linestyle='--', label='Mean Baseline')
plt.xlabel('Number of Estimators')
plt.ylabel('Mean Absolute Error')
plt.title('Random Forest: MAE vs. Number of Estimators')
plt.legend()
plt.tight_layout()
plt.show()

# recursive feature elimination

rf = RandomForestRegressor(n_estimators=100, random_state=3, n_jobs=1)
n_features_list = list(range(5, X_val.shape[1]+1, 5))
val_errors = []
train_errors = []
selected_features_dict = {}

for n_features in tqdm(n_features_list, desc='RFE Progress'):
    rfe = RFE(estimator=rf, n_features_to_select=n_features, step=5)
    rfe.fit(X_train, y_train)
    selected_features_dict[n_features] = list(X.columns[rfe.support_])
    X_train_rfe = rfe.transform(X_train)
    rf.fit(X_train_rfe, y_train)
    y_train_pred = rf.predict(X_train_rfe)
    train_errors.append(mean_absolute_error(y_train, y_train_pred))
    X_val_rfe = rfe.transform(X_val)
    y_val_pred = rf.predict(X_val_rfe)
    val_errors.append(mean_absolute_error(y_val, y_val_pred))

plt.figure(figsize=(8,5))
plt.plot(n_features_list, train_errors, label='Train MAE')
plt.plot(n_features_list, val_errors, label='Validation MAE')
plt.xlabel('Number of Selected Features')
plt.ylabel('Mean Absolute Error')
plt.title('RFE: MAE vs. Number of Features')
plt.legend()
plt.tight_layout()
plt.show()

# final model evaluation

final_features = selected_features_dict[20] 
rf_final = RandomForestRegressor(n_estimators=100, random_state=3, n_jobs=1)
X_train_final = X_train[:, [X.columns.get_loc(f) for f in final_features]]
rf_final.fit(X_train_final, y_train)

# predict on train, validation, and test sets
X_val_final = X_val[:, [X.columns.get_loc(f) for f in final_features]]
X_test_final = X_test[:, [X.columns.get_loc(f) for f in final_features]]

y_train_pred = rf_final.predict(X_train_final)
y_val_pred = rf_final.predict(X_val_final)
y_test_pred = rf_final.predict(X_test_final)

# calculate metrics
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_val = mean_absolute_error (y_val, y_val_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_val = r2_score(y_val, y_val_pred)
r2_test = r2_score(y_test, y_test_pred)

fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharex=True, sharey=True)
min_val = 0.0
max_val = 450.0

# train parity plot
axes[0].scatter(y_train, y_train_pred, alpha=0.5, color='blue')
axes[0].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[0].set_title(f'Train (MAE={mae_train:.3f}, R2={r2_train:.3f})')
axes[0].set_xlabel('True Formation Energy')
axes[0].set_ylabel('Predicted Formation Energy')
axes[0].set_aspect('equal', adjustable='box')

# validation parity plot
axes[1].scatter(y_val, y_val_pred, alpha=0.5, color='orange')
axes[1].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[1].set_title(f'Validation (MAE={mae_val:.3f}, R2={r2_val:.3f})')
axes[1].set_xlabel('True Formation Energy')
axes[1].set_ylabel('Predicted Formation Energy')
axes[1].set_aspect('equal', adjustable='box')

# test parity plot
axes[2].scatter(y_test, y_test_pred, alpha=0.5, color='green')
axes[2].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[2].set_title(f'Test (MAE={mae_test:.3f}, R2={r2_test:.3f})')
axes[2].set_xlabel('True Formation Energy')
axes[2].set_ylabel('Predicted Formation Energy')
axes[2].set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()


### Assignment 2

Go through an entire ML pipeline again, but this time use the Materials Project API to query a dataset that contains around 1-3k data points. Make the query more and more selective until only a few thousand entries are returned (e.g., by limiting the number of unique elements in the unit cell, limiting the band gap range, limiting the energy above hull value, etc.). Select one of the many possible target properties that the Materials Project has to offer (see homework 3, assignment 2, on how to get the information of all possible fields that can be requested and/or queried). 

Featurize the dataset you queried with a structural featurizer from `DScribe` and go through the remaining ML pipeline (similar to assignment 1). Summarize the performance metrics on the test set and upload the parity plots as a PNG files to the repository. 

In [ ]:
from mp_api.client import MPRester
import pandas as pd
from dscribe.descriptors import SOAP
from pymatgen.io.ase import AseAtomsAdaptor
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

api_key = 'ncNeAsZOmf7KOxBjJcP4O8O1NBrWciqx'

with MPRester(api_key) as mpr:
    data = data = mpr.materials.summary.search(theoretical=False, nelements= (4, 9))
    print(f'Size of dataset for materials containing 4-9 unique elements within unit cell: {len(data)}')
    data = data = mpr.materials.summary.search(theoretical=False, nelements= (5, 9))
    print(f'Size of dataset for materials containing 5-9 unique elements within unit cell: {len(data)}')
    data = mpr.materials.summary.search(theoretical=False, num_elements=(6, 9),
                                        fields=['material_id', 'band_gap', 'formula_pretty', 'structure'])
    print(f'Size of dataset for materials containing 6-9 unique elements within unit cell: {len(data)}')

df = pd.DataFrame([
    {"material_id": d.material_id,
     "formula": d.formula_pretty,
     "band_gap": d.band_gap,
     "structure": d.structure}
     for d in data
])

df.head()

species = list({str(sp) for s in df["structure"] for sp in s.symbol_set})
soap = SOAP(
    species=species,
    r_cut=5.0,
    n_max=4,
    l_max=3,
    average="inner",
    sparse=False
)

ase_adaptor = AseAtomsAdaptor()
features = []

for s in tqdm(df["structure"], desc="Featurizing structures"):
    atoms = ase_adaptor.get_atoms(s)
    feat = soap.create(atoms)
    features.append(feat)

X = np.array(features)
y = df["band_gap"].values
print(f"Feature shape: {X.shape}")

# feature scaling and dataset splitting

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

test_fraction = 0.1
validation_fraction = 0.2
X_trainval, X_test, y_trainval, y_test = train_test_split(X_scaled, y,
                                                          test_size=test_fraction,
                                                          random_state=3)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval,
                                                  test_size=validation_fraction/(1-test_fraction),
                                                  random_state=3)
print(f"Training fraction: {X_train.shape[0] / X_scaled.shape[0]:.2f}")
print(f"Validation fraction: {X_val.shape[0] / X_scaled.shape[0]:.2f}")
print(f"Test fraction: {X_test.shape[0] / X_scaled.shape[0]:.2f}")

# dummy baseline model

mean_train = y_train.mean()
baseline_mae = mean_absolute_error(y_val, [mean_train] * len(y_val))
print(f"Baseline MAE (predicting mean band gap): {baseline_mae:.4f} eV")

# hyperparameter optimization

n_estimators_list = [10, 50, 100, 500]
train_maes = []
val_maes = []

for n in tqdm(n_estimators_list, desc='Training RF models'):
    rf = RandomForestRegressor(n_estimators=n, random_state=3, n_jobs=1)
    rf.fit(X_train, y_train)
    y_train_pred = rf.predict(X_train)
    y_val_pred = rf.predict(X_val)
    train_maes.append(mean_absolute_error(y_train, y_train_pred))
    val_maes.append(mean_absolute_error(y_val, y_val_pred))

plt.figure(figsize=(8,5))
plt.plot(n_estimators_list, train_maes, marker='o', label='Train MAE')
plt.plot(n_estimators_list, val_maes, marker='o', label='Validation MAE')
plt.axhline(baseline_mae, color='gray', linestyle='--', label='Mean Baseline')
plt.xlabel('Number of Estimators')
plt.ylabel('Mean Absolute Error')
plt.title('Random Forest: MAE vs. Number of Estimators')
plt.legend()
plt.tight_layout()
plt.show()

# feature selection using random forest importances

rf = RandomForestRegressor(n_estimators=100, random_state=3, n_jobs=-1)
rf.fit(X_train, y_train)

# rank features by importance
importances = rf.feature_importances_
sorted_idx = np.argsort(importances)[::-1]

n_features_list = [5, 10, 25, 50, 100, 200, 400]  
train_errors, val_errors = [], []

for n_features in tqdm(n_features_list, desc="Evaluating top features"):
    top_idx = sorted_idx[:n_features]
    rf.fit(X_train[:, top_idx], y_train)
    y_train_pred = rf.predict(X_train[:, top_idx])
    y_val_pred = rf.predict(X_val[:, top_idx])
    train_errors.append(mean_absolute_error(y_train, y_train_pred))
    val_errors.append(mean_absolute_error(y_val, y_val_pred))

plt.figure(figsize=(8,5))
plt.plot(n_features_list, train_errors, marker='o', label='Train MAE')
plt.plot(n_features_list, val_errors, marker='o', label='Validation MAE')
plt.axhline(baseline_mae, color='gray', linestyle='--', label='Mean Baseline')
plt.xlabel('Number of Top Features Used')
plt.ylabel('Mean Absolute Error')
plt.title('Random Forest: MAE vs. Number of Top Features')
plt.legend()
plt.tight_layout()
plt.show()

# final model evaluation

# pick the number of top features with lowest validation MAE
best_n_idx = np.argmin(val_errors)
best_n_features = n_features_list[best_n_idx]
print(f"Best number of features based on validation MAE: {best_n_features}")

top_idx = sorted_idx[:best_n_features]

rf_final = RandomForestRegressor(n_estimators=100, random_state=3, n_jobs=-1)
rf_final.fit(X_train[:, top_idx], y_train)

y_train_pred = rf_final.predict(X_train[:, top_idx])
y_val_pred = rf_final.predict(X_val[:, top_idx])
y_test_pred = rf_final.predict(X_test[:, top_idx])

mae_train = mean_absolute_error(y_train, y_train_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_val = r2_score(y_val, y_val_pred)
r2_test = r2_score(y_test, y_test_pred)

# parity plots
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharex=True, sharey=True)
min_val, max_val = 0.0, 5.0

# train parity plot
axes[0].scatter(y_train, y_train_pred, alpha=0.5, color='blue')
axes[0].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[0].set_title(f'Train (MAE={mae_train:.3f}, R2={r2_train:.3f})')
axes[0].set_xlabel('True Band Gap (eV)')
axes[0].set_ylabel('Predicted Band Gap (eV)')
axes[0].set_aspect('equal', adjustable='box')

# validation parity plot
axes[1].scatter(y_val, y_val_pred, alpha=0.5, color='orange')
axes[1].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[1].set_title(f'Validation (MAE={mae_val:.3f}, R2={r2_val:.3f})')
axes[1].set_xlabel('True Band Gap (eV)')
axes[1].set_ylabel('Predicted Band Gap (eV)')
axes[1].set_aspect('equal', adjustable='box')

# test parity plot
axes[2].scatter(y_test, y_test_pred, alpha=0.5, color='green')
axes[2].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[2].set_title(f'Test (MAE={mae_test:.3f}, R2={r2_test:.3f})')
axes[2].set_xlabel('True Band Gap (eV)')
axes[2].set_ylabel('Predicted Band Gap (eV)')
axes[2].set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

### Assignment 3

Please, go through the first 6 pages of this paper: `Resources/Papers/Musil_Chemical_Review_2021_Review-of-Descriptors.pdf`. We discussed these aspects during the lectures, but please summarize any key insights that you may have missed during the lecture but understand better after reading the paper.

In [ ]:
# Paper is driven by a need to transform Cartesian coordinates of atoms into a suitable representation
# Descriptor/fingerprint are used interchangeably to indicate properties that are easier to compute than what you actually want to predict, but are strongly correlated to them
# SOAP is an example of a representation- precisely characterizes the instantaneous arrangement of atoms
    # Applies geometric and algebraic manipulations to Cartesian coordinates
    # Fulfills smoothness and symmetry with respect to isometries
# Space of features takes the form of vector space
    # May be simpler/more natural to describe the relationship between pairs of configurations
# Representations can be used with little modification for any atomistic system
# QSPR is quantitative structure-property relationship
    # This can be compared to bottom-up modeling, these are different from machine-learned potentials
# ML tasks can benefit from being based on local features, rather than global
# There are many trivial symmetries- this can lead to inefficiency
    # A Z-matrix is a collection of internal coordinates that is sufficient to fully characterize structure geometry
        # Has limitations- bonding patterns will change in the simulation of a chemically active system
# Majority of atomic-scale properties are continuous, smooth functions of atomic coordinates
    # Features and internal coordinates are usually smooth
    # The Coulomb matrix helps to obtain permutation invariance
# We can assume an additive decomposition of properties to make problems simpler